# Code for sequence and structural alignment of related proteins

In [2]:
import numpy as np

# BioPython 
from Bio.Seq import Seq
from Bio import AlignIO, SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Blast import NCBIXML


import os
import subprocess
import sys, requests

In [3]:
# Bokeh imports
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot
from bokeh.plotting import output_file, save

## 1) Sequence alignment 
BLAST-Mafft-Bokeh implementation

---This implementation needs the installation of both Bokeh and also the command-line mafft program---

In [4]:
# Defining colors for each protein residue
def get_colors_protein(seqs):
    """Make colors for bases in sequence

    Args:
        seqs (list, str): List or string with protein sequence

    Returns:
        list: List with colors
    """
    text = [i for s in list(seqs) for i in s]
    aa_colors = {
    'A': 'red',    # Alanine
    'R': 'blue',   # Arginine
    'N': 'green',  # Asparagine
    'D': 'yellow', # Aspartic acid
    'C': 'orange', # Cysteine
    'Q': 'purple', # Glutamine
    'E': 'cyan',   # Glutamic acid
    'G': 'magenta',# Glycine
    'H': 'pink',   # Histidine
    'I': 'brown',  # Isoleucine
    'L': 'gray',   # Leucine
    'K': 'lime',   # Lysine
    'M': 'teal',   # Methionine
    'F': 'navy',   # Phenylalanine
    'P': 'olive',  # Proline
    'S': 'maroon', # Serine
    'T': 'silver', # Threonine
    'W': 'gold',   # Tryptophan
    'Y': 'skyblue',# Tyrosine
    'V': 'violet', # Valine
    '-':'white'
    }
    colors = [aa_colors[i] for i in text]
    return colors

In [5]:
def view_alignment_bokeh(aln, fontsize="9pt", plot_width=800, file_name='alignment'):
    """Bokeh sequence alignment view
    From: https://dmnfarrell.github.io/bioinformatics/bokeh-sequence-aligner"""
    # The function takes a biopython alignment object as input.
    # rec is the alignment record: Each one of the entries given as input
    seqs = [rec.seq for rec in (aln)] # Each sequence input
    ids = [rec.id for rec in aln] # Each entry ID
    text = [i for s in list(seqs) for i in s] #Al units joind on same list
    # List with ALL colors
    colors = get_colors_protein(seqs)    
    N = len(seqs[0]) # What if they're not the same length???
    S = len(seqs)    
    width = .4

    x = np.arange(1,N+1)
    y = np.arange(0,S,1)
    #creates a 2D grid of coords from the 1D arrays
    xx, yy = np.meshgrid(x, y)
    #flattens the arrays
    gx = xx.ravel()
    gy = yy.flatten()
    #use recty for rect coords with an offset
    recty = gy+.5 # Just to make the rectangles twice the size and the letter in the middle
    h= 1/S
    #now we can create the ColumnDataSource with all the arrays
    print(f'Aligning {S} sequences of lenght {N}')
    # ColumnDataSource is a JSON dict that maps names to arrays of values
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs)*10+50
    x_range = Range1d(0, N+1, bounds='auto') # (start, end)
    if N>150:
        viewlen=150
    else:
        viewlen=N
    #view_range is for the close up view
    view_range = (0,viewlen)
    tools="xpan, xwheel_zoom, reset, save"

    #entire sequence view (no text, with zoom)
    p = figure(title=None, width=plot_width, height=plot_height,
               x_range=x_range, y_range=(0,S), tools=tools,
               min_border=0, toolbar_location='below')
    # Rect simply places rectangles of wifth "width" into the positions defined by x and y
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                 line_color=None, fill_alpha=0.6)
    # Source does mapping from keys in rects to values in ColumnDataSource definition
    p.add_glyph(source, rects) 
    p.yaxis.visible = False
    p.grid.visible = False  

    #sequence text view with ability to scroll along x axis
    p1 = figure(title=None, width=plot_width, height=plot_height,
                x_range=view_range, y_range=ids, tools="xpan,reset",
                min_border=0, toolbar_location='below')#, lod_factor=1)   
    # Text does the same thing as rectangles but placing letter (or words) instead, aligned accordingly   
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black",
                text_font="monospace",text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p1.add_glyph(source, glyph)
    p1.add_glyph(source, rects)

    p1.grid.visible = True
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0

    p = gridplot([[p],[p1]], toolbar_location='below')

    output_file(filename=f"{file_name}.html", title="Alignment result")
    save(p)
    
    return p

Testing alignment with two sequences of equal length

In [6]:
# Note here: The aminoacid sequences must be the same length, with missing units represented as '-'
aln = AlignIO.read('covid.aln','fasta')
p = view_alignment_bokeh(aln, plot_width=1500, file_name='simple_alignment')
# pn.pane.Bokeh(p)

Aligning 2 sequences of lenght 301


ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : text_font='monospace' [no close matches] {renderer: GlyphRenderer(id='p1062', ...)}


### Sequence alignment of BLAST sequences
*Input reference sequence on BLAST to query for similar sequences, select seqs in the output to finally generate an HTML file with multi seq alignment*

In [7]:
def parse_blast(results_file, verbose):
    # Parse results
    E_VALUE_THRESH = 1e-20 

    matches_seq = []
    matches_id = []
    scores = []

    for record in NCBIXML.parse(open(results_file)): 
        if record.alignments: 
            if verbose:
                print("\n") 
                print("query: %s" % record.query[:100]) 
            query_seqs = []
            query_ids = []
            query_scores = []
            for align in record.alignments: 
                for hsp in align.hsps: 
                    if hsp.expect < E_VALUE_THRESH: 
                        # Print sequence identity, title, and gapless sequence substring that aligns
                        hsps0 = align.hsps[0]
                        sequence_to_model = hsps0.sbjct.replace('-','')
                        pidentity = round(100.0 * hsps0.identities / (hsps0.query_end-hsps0.query_start+1), 2)
                        if verbose:
                            print(f'length {hsps0.identities}, score {pidentity}: {align.title}')# {sequence_to_model}')
                        query_seqs.append(sequence_to_model)
                        query_ids.append(align.title)
                        query_scores.append(pidentity)
        matches_seq.append(query_seqs)
        matches_id.append(query_ids)
        scores.append(query_scores)

    return matches_seq, matches_id, scores

def get_blast_seqs(seq_source, input_type="fasta", save_file="results.xml", nalign=500, database="refseq_protein", verbose=True):
    """Run a BLAST search on a protein sequence.
    Args:
        seq_source (string, list): Source with the sequence.
        input_type (str, optional): Type of sequence source ["pre-cal", "fasta", "sequence"]. Defaults to "fasta".
        save_file (str, optional): Name of output file storing BLAST results. Defaults to "results.xml".

    Returns:
        (list, list): Matched sequences, Matched IDs
    """
    
    if input_type == "pre-calc":
        matches_seq, matches_id, scores = parse_blast(seq_source, verbose)
        return matches_seq, matches_id, scores
    elif input_type == "fasta":
        # Input is file name
        sequence = open(seq_source).read() 
    elif input_type == "sequence":
        # Input is sequence
        sequence = seq_source
    else: # Another source?
        return
    
    # Retrieve blastp results
    program = 'blastp' # protein sequence BLAST
    database =  database # non-redundant protein database
    alignments = nalign # number of alignments to retrieve
    result_handle = NCBIWWW.qblast(program, database, sequence, alignments=alignments) 

    with open(save_file, 'w') as file: 
        blast_results = result_handle.read() 
        file.write(blast_results)

    matches_seq, matches_id, scores = parse_blast(save_file, verbose)

    return matches_seq, matches_id, scores

def process_align_data(input_alignment, output_file):
    ''' Modify sequences in multi-seq alignment to remove gap characters '-' 
    '''
    alignment = SeqIO.parse(input_alignment, "fasta")
    filtered_sequences = []
    for rec in alignment:
        # Remove gap characters '-' from the sequence
        print(rec.seq)
        filtered_sequence = ''.join(char for char in rec.seq if char != '-')#rec.seq.ungap("-")
        # Update SeqRecord with the filtered sequence
        filtered_seq_record = SeqRecord(filtered_sequence, id=rec.id, description=rec.description)
        filtered_sequences.append(filtered_seq_record)

    SeqIO.write(filtered_sequences, output_file, "fasta")
    return output_file

In [8]:
# Calculate homologous sequences (Comment if file has already been generated because this takes some time.
# matches_seq, matches_id = get_blast_seqs('covid.aln', input_type="fasta", save_file="results.xml", nalign=500, database="refseq_protein")

# Retrieve pre-computed file
matches_seq, matches_id, scores = get_blast_seqs("results.xml", input_type="pre-calc")



query: MERS-CoV Mpro
length 301, score 100.0: ref|YP_007188578.1| ORF1a polyprotein [Betacoronavirus England 1]
length 301, score 100.0: ref|YP_009047203.1| 1A polyprotein [Middle East respiratory syndrome-related coronavirus]
length 301, score 100.0: ref|YP_007188577.3| ORF1ab polyprotein [Betacoronavirus England 1]
length 301, score 100.0: ref|YP_009047202.1| 1AB polyprotein [Middle East respiratory syndrome-related coronavirus]
length 301, score 100.0: ref|YP_009047217.1| nsp5 protein [Middle East respiratory syndrome-related coronavirus] >ref|YP_009047233.1| nsp5 protein [Middle East respiratory syndrome-related coronavirus] >ref|YP_009944285.1| nsp5 [Betacoronavirus England 1] >ref|YP_009944296.1| nsp5 [Betacoronavirus England 1]
length 288, score 95.68: ref|YP_009361855.1| ORF1a polyprotein [Bat coronavirus]
length 288, score 95.68: ref|YP_009361856.2| ORF1ab polyprotein [Bat coronavirus]
length 248, score 82.39: ref|YP_009944343.1| nsp5 [Pipistrellus bat coronavirus HKU5] >ref

Select sequences to visualize from a checklist with BLAST results (example for MERS-CoV)

In [9]:

from ipywidgets import Layout, Checkbox, Box

items_layout = Layout(flex='1 1 auto',
                      width='auto')    

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='',
                    width='80%')

# We will select BLAST matches for the first sequence in the input fasta file (MERS-Cov)
data = [(' '.join(matches_id[0][s].split(' ')[1:])) for s in range(len(matches_id[0]))]
ref_nums = [(''.join(matches_id[0][s].split(' ')[0])) for s in range(len(matches_id[0]))]
items = [Checkbox(description=w, layout=items_layout) for w in data]
output = Box(children=items, layout=box_layout)
print("Choose which proteins you want to compare with the REF: \n")
display(output)

Choose which proteins you want to compare with the REF: 



Box(children=(Checkbox(value=False, description='ORF1a polyprotein [Betacoronavirus England 1]', layout=Layout…

Comparing the chosen sequences

In [108]:
# Add selected sequences to array
# Include the Reference sequence for comparison
selected_labels = [ref_nums[0]]#['REF']
selected_description= [items[0].description]
selected_seqs = [matches_seq[0][0]]

for i in range(0, len(items)):
    if items[i].value == True:
        selected_seqs.append(matches_seq[0][i])
        selected_labels.append(ref_nums[i])
        selected_description.append(items[i].description)
records = []
for r in range(len(selected_labels)):
    rec = SeqRecord(Seq(selected_seqs[r]), id=selected_labels[r],  description=selected_description[r])
    records.append(rec)

# Run alignment with MAFFT
input_file = "covid_BLAST.aln"
out_file = "output.fasta"
SeqIO.write(records, input_file, "fasta")
cmd = f"mafft {input_file} > {out_file}"
subprocess.run(cmd, shell=True)

align = AlignIO.read(out_file, "fasta")

# Generate output file without gaps for later steps
clean_output = process_align_data(out_file, "blast_selection.fasta")

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0

Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.



In [109]:
p = view_alignment_bokeh(align, plot_width=1500, file_name='alignment_blast') 

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : text_font='monospace' [no close matches] {renderer: GlyphRenderer(id='9bc27cd3-3eb6-4c99-ae69-5b7a48e566f3', ...)}


Aligning 1 sequences of lenght 301


## 2) Structure alignment pipeline   
Retrieve pdb record for selection --> Alphafold setup for seqs not in pdb database --> Structure alignment

Retrieve PDB for chosen sequences

In [14]:
def get_seq_alphafold(seq, seq_name, file_dir):
    ''' Generate alphafold structure of a given sequence'''
    print("Alphafold has not been implemented")

    # Write CSV file for the sequence
    import csv
    csv_file_name = file_dir + "mers_input.csv"
    with open(csv_file_name, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        # Write the header
        writer.writerow(["id", "sequence"])
        # Write the sequence
        writer.writerow([seq_name + "_{}", seq])
    
        return csv_file_name
    
def retrieve_existing_alphafold(pdb_id):
    """_summary_

    Args:
        pdb_id (_type_): _description_
    """
    # First convert to uniprot
    uniprot_id = pdb_to_uniprot(pdb_id)
    uniprot_id = uniprot_id.lower()
    fjson = request_alphafold(uniprot_id)

    return fjson

def request_rcsb_pdbid(pdb_id):
    """ A function to request a protein entry from the rcsb API with a pdb ID
    """
    requestURL = f"https://data.rcsb.org/rest/v1/core/entry/{pdb_id}"
    r = requests.get(requestURL, headers={ "Accept" : "application/json"})
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    return r.json()

def request_alphafold(uniprot_id):
    """
    A function to request a protein entry from the UniProt API (from avidome-analysis)
    """
    requestURL = f"https://alphafold.ebi.ac.uk/api/uniprot/summary/{uniprot_id}.json"
    print(requestURL)
    r = requests.get(requestURL)
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    return r.json()

def pdb_to_uniprot(pdb_id):
    """ With the API, will find the corresponding Uniprot ID of a PDB entry
    """
    import sys, requests
    requestURL = f"https://www.ebi.ac.uk/pdbe/api/mappings/uniprot/{pdb_id}"

    r = requests.get(requestURL, headers={ "Accept" : "application/json"})
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    
    fjson = r.json()
    uniprot_dic = fjson[pdb_id]['UniProt']
    uniprot_id = next(iter(uniprot_dic))
    return uniprot_id

def check_resolution(pdb_id):
    fjson = request_rcsb_pdbid(pdb_id)
    return fjson['rcsb_entry_info']['resolution_combined'][0]

def choose_best_pdb_entry(hits, best_match_percent):
    ''' If a sequence have different PDB files all with the same alignment match, we choose and retrieve the one with the highest resolution'''
    max_res = 10 # some unrealistically large number?
    best_pdb_record = ""
    for h in hits:
        id_match = hits[h]['percent_identity']
        if id_match == best_match_percent: 
            res = check_resolution(h)
            if res < max_res:
                best_pdb_record = h
                max_res = res
        else: # We're only interested in the entries with the max possible alignment
            break
    print(f"The best PDB entry is {best_pdb_record}, with match {best_match_percent}% and res {max_res}A")
    return best_pdb_record

In [15]:
class PDB_record():
    def __init__(self, label:str, query_seq:str, description:str, chain:int) -> None:
        self.label = label
        self.description = description
        self.seq = query_seq
        self.chain = chain


def retrieve_pdb(seq_file, blast_file="results.xml", pre_gen=False, min_id_match=99, pdb_folder="pdb_search_bkp/"):
    ''' Retrieve the PDB record of a given sequence. Alternative solution to prody'''
    from pathlib import Path
    import prody

    folder_for_bkps = pdb_folder
    Path(folder_for_bkps).mkdir(parents=True, exist_ok=True)
    record_name = f"{folder_for_bkps}{blast_file}"

    if pre_gen: # the search was done before 
        # Check if the file exists
        record_path = Path(record_name)
        if not record_path.exists():
            raise ValueError("pre_gen was set to True but the file does not exist. Set to False to generate the record")
    else:
        # Find pdb matching given sequence
        matches_seq, matches_id, scores = get_blast_seqs(seq_file, input_type="fasta", save_file=record_name, nalign=500, database="pdb", verbose=False)
        print("saving in ", record_name)

    # Load original sequence names and descriptors for reference
    pdb_file_record = []
    for seq_record in SeqIO.parse(seq_file, "fasta"):
        seq_id = seq_record.id
        seq_description = seq_record.description
        seq_chain = int(seq_record.id.split('|')[1].split('.')[-1])
        pdb_file_record.append(PDB_record(seq_id, seq_record.seq, seq_description, seq_chain))

    # Load data 
    matches_seq, matches_id, scores = get_blast_seqs(record_name, input_type="pre-calc", verbose=False)
    # best_ids = matches_id[:,0]
    best_scores = [max(s) for s in scores]
    # print(f"The best matches are '{best_ids}' with scores {best_scores}")
    
    match_hits = parse_pdb_blast_results(matches_seq, matches_id, scores, min_score=min_id_match)

    for i, hits in enumerate(match_hits):
        if len(hits) == 0: # The record doesn't have a PDB with high confidence
            filename = get_seq_alphafold(pdb_file_record[i].seq, 'some_seq', folder_for_bkps)
            pdb_file_record[i].pdb_file = filename
            pdb_file_record[i].pdb_id = None
            
        else:
            # Return the pdb entry with the max alignment and max resolution (if there are multiple matches)
            best_pdb_record = choose_best_pdb_entry(hits, best_scores[i])
            filename = prody.fetchPDB(best_pdb_record, folder=folder_for_bkps)
            subprocess.run(["gunzip", filename])
            pdb_file_record[i].pdb_file = filename
            pdb_file_record[i].pdb_id = best_pdb_record

    return pdb_file_record


def parse_pdb_blast_results(blast_seq, blast_ids, blast_scores, min_score):
    ''' For a pdb database search extract pdb ids with a minimum score (Temp fix to prody).
        Outputs are lists because iterables are needed for functions down the pipeline.
    '''

    assert len(blast_ids) == len(blast_scores) == len(blast_seq)

    match_hits = []
    for q in range(len(blast_ids)): # Loop over queries
        hits = {}
        for e in range(len(blast_ids[q])): # Loop over BLAST entries
            if blast_scores[q][e] >= min_score:
                raw_id = blast_ids[q][e].split('|')
                pdb_index = raw_id.index('pdb')
                pdb_id = raw_id[pdb_index + 1]

                hits[pdb_id] = {}
                hits[pdb_id]['percent_identity'] = blast_scores[q][e]
                hits[pdb_id]['sequence'] = blast_seq[q][e]

        match_hits.append(hits)

    return match_hits

The next block contains failed attempts of doing a BLAST search for uniprot IDs. The goal is to retrieve records from Alphafold database, which are registered with an UniprotID.

In [16]:
def map_swissprot_to_uniprot(swissprot_id):
    # Construct the URL for UniProt ID mapping service
    url = f"https://www.ebi.ac.uk/proteins/api/features?accession={swissprot_id}"
    
    r = requests.get(url, headers={ "Accept" : "application/json"})
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    
    fjson = r.json()
    #uniprot_dic = fjson[pdb_id]['UniProt']
    #uniprot_id = next(iter(uniprot_dic))
    return fjson #uniprot_id

seq_file = "test.fasta"
# matches_seq, matches_id, scores = get_blast_seqs(seq_file, input_type="fasta", save_file="pdb_search_bkp/results_uniprot.xml", nalign=500, database="swissprot", verbose=False)

# mapped_uniprot_ids = map_swissprot_to_uniprot("K9N638")[0]
# print("Mapped UniProt IDs:", mapped_uniprot_ids)


 **NOTE** Manual implementation of PDB search was implemented because ProDy has problems with retrieving data. I'm still using ProDy to retrieve PDB files (`prody.fetchPDB`). If we're keeping this solution, it may be worth it to implement that by hand too (to minimize unnecessary package imports)

The best PDB entry (if any) for each alignment sequence is saved on the provided `pdb_folder`.

In [17]:
clean_output = "blast_selection.fasta"
pdb_file_record = retrieve_pdb(clean_output, blast_file="results_pdb.xml", pre_gen=True, min_id_match=99, pdb_folder="pdb_search_bkp/")

@> PDB file is found in working directory (pdb_search_bkp/7xry.pdb).


The best PDB entry is 7XRY, with match 100.0% and res 1.99A


gunzip: pdb_search_bkp/7xry.pdb: unknown suffix -- ignored
@> PDB file is found in working directory (pdb_search_bkp/7xry.pdb).


The best PDB entry is 7XRY, with match 100.0% and res 1.99A
Alphafold has not been implemented
Alphafold has not been implemented


gunzip: pdb_search_bkp/7xry.pdb: unknown suffix -- ignored
@> PDB file is found in working directory (pdb_search_bkp/6yb7.pdb).


The best PDB entry is 6YB7, with match 100.0% and res 1.25A
Alphafold has not been implemented


gunzip: pdb_search_bkp/6yb7.pdb: unknown suffix -- ignored
@> PDB file is found in working directory (pdb_search_bkp/6xhl.pdb).


The best PDB entry is 6XHL, with match 100.0% and res 1.471A


gunzip: pdb_search_bkp/6xhl.pdb: unknown suffix -- ignored


The sequence label and descriptor, query sequence, and pdb_id found in the database, are stored in the `PDB_record` class.

In [18]:
for rec in pdb_file_record:
    if rec.pdb_id:
        print(f"For the sequence {rec.description}: {rec.seq[:20]}..., a PDB was found matching chain {rec.chain} and saved in {rec.pdb_file}.")
    else:
        print(f"For the sequence {rec.label}: {rec.seq[:20]}..., no PDB entry was found. Alphafold csv saved in {rec.pdb_file}.")

For the sequence ref|YP_007188578.1| ORF1a polyprotein [Betacoronavirus England 1]: SGLVKMSHPSGDVEACMVQV..., a PDB was found matching chain 1 and saved in pdb_search_bkp/7xry.pdb.
For the sequence ref|YP_009047203.1| 1A polyprotein [Middle East respiratory syndrome-related coronavirus]: SGLVKMSHPSGDVEACMVQV..., a PDB was found matching chain 1 and saved in pdb_search_bkp/7xry.pdb.
For the sequence ref|YP_009944343.1|: SGLVKMAAPSGVVENCMVQV..., no PDB entry was found. Alphafold csv saved in pdb_search_bkp/mers_input.csv.
For the sequence ref|YP_009944337.1|: SGLVKMAAPSGVVENCMVQV..., no PDB entry was found. Alphafold csv saved in pdb_search_bkp/mers_input.csv.
For the sequence ref|YP_009725301.1| 3C-like proteinase [Severe acute respiratory syndrome coronavirus 2] >ref|YP_009742612.1| 3C-like proteinase [Severe acute respiratory syndrome coronavirus 2]: SGFRKMAFPSGKVEGCMVQV..., a PDB was found matching chain 1 and saved in pdb_search_bkp/6yb7.pdb.
For the sequence ref|YP_009944265.1|: SGI

For the sequences with no associated PDB entry, 
right now the `get_seq_alphafold` function only generates the input csv file with the sequence, and ColabFold is run in lilac. Will have to integrate with the lilac workflow later.

In [19]:
csv_file = get_seq_alphafold(pdb_file_record[1].seq, "MERS_CoV", "pdb_search_bkp/")
print(f"Copy the files {csv_file} and {pdb_file_record[1].pdb_file} to the ColabFold working directory")


Alphafold has not been implemented
Copy the files pdb_search_bkp/mers_input.csv and pdb_search_bkp/7xry.pdb to the ColabFold working directory


Asumming we already ran ColabFold and the file outputs are in the folder `pdb_search_bkp/`, we select the result with less RMSD compared to the reference.  
The following blocks were also run on lilac, because OpenEye doesn't work on my Jupyter at the moment.

In [2]:
from asapdiscovery.data.openeye import load_openeye_pdb, save_openeye_pdb
from asapdiscovery.modeling.modeling import superpose_molecule

def rmsd_alignment(result_pdb, ref_pdb, chain, final_pdb):
    """Calculate RMSD of ColabFold PDB output against a reference 

    Args:
        result_pdb (str): The path to the mobile pdb
        ref_pdb (str): The path to the reference pdb
        chain (str): Label of chain to align
        final_pdb (str): Path to the file to be saved

    Returns:
        _type_: _description_
    """
    protein = load_openeye_pdb(result_pdb)
    ref_protein = load_openeye_pdb(ref_pdb)

    aligned_protein, rmsd = superpose_molecule(ref_protein, protein, ref_chain=chain, mobile_chain=chain)
    pdb_aligned = save_openeye_pdb(aligned_protein, final_pdb)

    return rmsd, pdb_aligned

def select_best_colabfold(results_dir, seq_name, pdb_ref, chain="A", final_pdb="aligned_protein.pdb"):
    """Select the best output (repetition) from the ColabFold run based on its RMSD wrt the reference.

    Args:
        results_dir (str): The directory containing the ColabFold results.
        seq_name (str): The name we gave to the sequence in the csv file.
        pdb_ref (str): The path to the reference pdb.
        chain (str): Label of chain to align.
        final_pdb (str): Path to the file to be saved.

    Returns:
        _type_: _description_
    """

    from pathlib import Path
    colab_fold_dir = results_dir

    rmsds = []
    seeds = []
    file_seed = []
    for file_path in Path(colab_fold_dir).glob(seq_name+"_*_unrelaxed_rank_001_alphafold2_ptm_model_1_seed_*.pdb"):
        pdb_to_compare = file_path
        seed = str(pdb_to_compare).split('_')[-1].split('.')[0]
        rmsd, pdb = rmsd_alignment(pdb_to_compare, pdb_ref, chain, final_pdb)
        rmsds.append(rmsd)
        seeds.append(seed)
        file_seed.append(file_path)
        print(f"RMSD for seed {seed} is {rmsd}A")

    min_rmsd = np.argmin(rmsd)
    min_rmsd_file = file_seed[min_rmsd]
    print(f"Seed with less RMSD is {seeds[min_rmsd]} with RMSD {rmsds[min_rmsd]}")

    min_rmsd, final_pdb = rmsd_alignment(min_rmsd_file, pdb_ref, chain, final_pdb)

    return final_pdb, min_rmsd

In [ ]:
colab_fold_dir = "pdb_search_bkp/"
pdb_ref = pdb_file_record[1].pdb_file
chain = ['A', 'B'][pdb_file_record[1].chain - 1]

# Running the function on Jupyter makes the Kernel crash
seq_name = "MERS_Mpro" 
# alphafold_pdb, min_rmsd = select_best_colabfold(colab_fold_dir, seq_name, pdb_ref, final_pdb="aligned_protein.pdb")